# DEMO TOOL TRADITIONAL IRISH MUSIC CLASSIFIER FOR SCORES

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elloza/ITMA-classifier-paper/blob/main/demo/DemoTool.ipynb)


⚠️ **YOU MUST SELECT THE PRO EXECUTION ENVIRONMENT TO RUN THE DEMO
(Jukemir need a lot of RAM and VRAM)**⚠️


This is the demo tool for deploying the traditional Irish music classifier for scores.

<!-- Align center and with space in the middle -->
<div align="center" style="margin: 0px 0px 0px 0px;">

  <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA" style="margin:20px;">
    <img src="https://usal.es/files/logo_usal.png" alt="Logo" width="250" height="100" style="margin:10px;padding:20px;">
  </a>

  <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA" style="margin:20px;">
    <img src="https://www.itma.ie/wp-content/themes/ITMA/images/itma-logo.svg" alt="Logo" width="250" height="100" style="margin:10px;padding:20px;">
  </a>

</div>

In [1]:
from IPython.display import display, HTML, clear_output

def clear(message):
    clear_output(wait=True)
    display(HTML(f'<span style="color: green;">{message}</span>'))

clear("Completed")

In [11]:
!nvidia-smi

Thu Feb 15 10:54:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Clone Repository on colab

In [2]:
!git clone https://github.com/elloza/ITMA-classifier-paper

Cloning into 'ITMA-classifier-paper'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 10), reused 31 (delta 8), pack-reused 0
Receiving objects: 100% (37/37), 9.12 MiB | 19.09 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [3]:
%cd ITMA-classifier-paper
%cd demo

/content/ITMA-classifier-paper
/content/ITMA-classifier-paper/demo


## Install all the required libraries

This could take some time.⏳

In [4]:
# Install pycaret
!pip install pycaret[full]
# install gradio
!pip install -U gradio
clear("Libraries installed")

Install Conversion File Libraries

In [5]:
%pip install midi2audio
!sudo apt-get update && sudo apt-get install fluidsynth -y && sudo apt-get install abcmidi -y
%pip install git+https://github.com/SpotlightKid/abc2xml
clear("Converters installed")

Install MERT Dependencies

In [6]:
# Jukemir
%pip install git+https://github.com/rodrigo-castellon/jukemirlib.git

# Mule
!sudo apt-get install git-lfs
!git clone https://github.com/PandoraMedia/music-audio-representations.git ~/.cache/music-audio-representations
%cd ~/.cache/music-audio-representations
!git lfs pull
%cd
%pip install --upgrade pip
%pip install "cython<3.0.0" wheel && pip install pyyaml==5.4.1 --no-build-isolation
%pip install librosa==0.9.2
%pip install sxmp-mule

# Mert
%pip install transformers==4.29.2
%pip install datasets
%pip install nnAudio==0.3.1
%pip uninstall -y tqdm
%pip install -U tqdm
clear("Mule Mert and Jukemir installed")

In [9]:
%cd /content/ITMA-classifier-paper/demo

/content/ITMA-classifier-paper/demo


# Execute Gradio from here

In [10]:
!pwd

/content/ITMA-classifier-paper/demo


In [ ]:
import pycaret
from pycaret.classification import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import shutil

from MusicFileConverter import MusicFileConverter
from EmbeddingExtractor import EmbeddingExtractor

# GENRES
N_GENRES = 15

# Definir la carpeta de subidas y asegurarse de que exista
uploads_folder = "./uploads/"
os.makedirs(uploads_folder, exist_ok=True)

# LOAD CLASSIFIER
classifier = load_model("embedding_mert_best_model")

# LOAD EMBEDDING EXTRACTOR (CHANGE THIS TO OTHER MODELS LIKE MULE OR JUKEMIR)
extractor = EmbeddingExtractor("mert")

print("Loaded embedding model: ")
print(extractor.embedding_model)

# TODO: CHANGE PATHS!
converter = MusicFileConverter(abc2xml_path_file="./abc2xml.py", xml2abc_path_file="./xml2abc.py")

def process_file(fileobj, folder):
    # Asegurarse de que el nombre del archivo no contenga rutas para evitar problemas de seguridad
    filename = os.path.basename(fileobj.name)
    path = os.path.join(folder, filename)
    shutil.copyfile(fileobj.name, path)
    return path

def transform_file_to_wav(uploaded_file):

    print(uploaded_file)

    if uploaded_file is None:
      raise Exception("File is None")

    file_path = process_file(uploaded_file, uploads_folder)

    print("Fichero escrito:"+file_path)

    # Obtener el nombre del fichero sin extensión
    nombre_fichero_sin_extension = os.path.splitext(os.path.basename(file_path))[0]

    wav_file = os.path.join(os.path.dirname(file_path), nombre_fichero_sin_extension + '.wav')

    print(wav_file)

    # Check if file is abc, mid, or xml
    if file_path.endswith(".abc"):
        # Crear rutas para los archivos .mid y .wav
        midi_file = uploads_folder + nombre_fichero_sin_extension + '.mid'
        midi_file = converter.abc_to_midi(file_path, midi_file)
        wav_file = converter.midi_to_wav(midi_file, wav_file)
    elif file_path.endswith(".mid"):
        wav_file = converter.midi_to_wav(file_path, wav_file)
    elif file_path.endswith(".xml"):
        midi_file = uploads_folder + nombre_fichero_sin_extension + '.mid'
        midi_file = converter.xml_to_midi(file_path,midi_file)
        wav_file = converter.midi_to_wav(midi_file, wav_file)
    else:
        raise Exception("File format not supported")

    return wav_file

def transform_wav_to_embedding_row(wav_file):
    # Extract embedding from a music file
    embedding = extractor.get_embedding(wav_file)
    print("Mert embedding: ")
    print(embedding)

    # Crear nombres de columnas basados en la longitud del embedding
    column_names = [f"embedding_mert_{i}" for i in range(len(embedding))]

    # Convertir el embedding en un dataframe y retornar
    embedding_df = pd.DataFrame([embedding], columns=column_names)
    return embedding_df

def classify_file(uploaded_file):
    # Supongamos que nuestro modelo nos da scores para tres clases: A, B y C
    # Estos son valores de ejemplo
    global classifier
    global converter

    # 1 - Transform abc, mid, or xml file to wav file
    wav_file = transform_file_to_wav(uploaded_file)
    print(wav_file)

    # 2 - Transform wav file to embedding row with columns embedding_mule_0, embedding_mule__1, etc
    df = transform_wav_to_embedding_row(wav_file)

    # 3 - Perform the inference
    pred = predict_model(classifier, df, raw_score=True)

    print(pred)

    # Get only the columns called prediction_score_*
    pred = pred.filter(regex="prediction_score_*")

    print(pred)

    # Get pred as a dictionary where keys are the columns names and values are the columns values
    pred_dict = pred.to_dict("records")[0]

    # Rename the keys of the dictionary to remove the prefix prediction_score_
    pred_dict = {key.replace("prediction_score_", ""): value for key, value in pred_dict.items()}

    return pred_dict


import gradio as gr

# Logo y título personalizado utilizando HTML
title_with_logo = """
    <div style='text-align: center;'>
        <div style='margin-bottom: 10px; font-weight: bold;'>
            IRISH TRADITIONAL MUSIC GENRE CLASSIFICATION
        </div>
        <div style='margin-bottom: 10px; font-weight: regular;'>
            Identifying Irish traditional music genres using latent audio representations
        </div>
        <div style='display: flex; justify-content: space-between; align-items: center;'>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://usal.es/files/logo_usal.png" alt="Logo" width="150" height="50" style="margin:10px;padding:20px;">
            </a>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://www.itma.ie/wp-content/themes/ITMA/images/itma-logo.svg" alt="Logo" width="150" height="50" style="margin:10px;padding:20px;">
            </a>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://cordis.europa.eu/images/logo/logo-ec-es.svg" alt="Logo" width="150" height="50" style="margin:10px;padding:20px;">
            </a>
        </div>
    </div>
"""

with gr.Blocks(title="ITMA Classifier", theme=gr.themes.Default(primary_hue="emerald")) as demo:
    iface = gr.Interface(
        fn=classify_file,
        inputs=gr.File(label="Upload .abc, .mid, or .xml file"),
        outputs=gr.Label(num_top_classes=N_GENRES),
        live=False,
        title=title_with_logo,
        allow_flagging="never"
    )

# Share the demo with a public link
demo.launch(share=True, inbrowser=True, debug=True)

## Execute the application

## Deploy Gradio Application

# JUKEMIR

Same code but with jukemir

This could take some time.⏳ **30 minutes aprox for the first inference (the jukemir models download takes time)**


In [ ]:
import pycaret
from pycaret.classification import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import shutil

from MusicFileConverter import MusicFileConverter
from EmbeddingExtractor import EmbeddingExtractor

# GENRES
N_GENRES = 15

# Definir la carpeta de subidas y asegurarse de que exista
uploads_folder = "./uploads/"
os.makedirs(uploads_folder, exist_ok=True)

# LOAD CLASSIFIER
classifier = load_model("embedding_jukemir_best_model")

# LOAD EMBEDDING EXTRACTOR (CHANGE THIS TO OTHER MODELS LIKE MULE OR JUKEMIR)
extractor = EmbeddingExtractor("jukemir")

print("Loaded embedding model: ")
print(extractor.embedding_model)

# TODO: CHANGE PATHS!
converter = MusicFileConverter(abc2xml_path_file="./abc2xml.py", xml2abc_path_file="./xml2abc.py")

def process_file(fileobj, folder):
    # Asegurarse de que el nombre del archivo no contenga rutas para evitar problemas de seguridad
    filename = os.path.basename(fileobj.name)
    path = os.path.join(folder, filename)
    shutil.copyfile(fileobj.name, path)
    return path

def transform_file_to_wav(uploaded_file):

    print(uploaded_file)

    if uploaded_file is None:
      raise Exception("File is None")

    file_path = process_file(uploaded_file, uploads_folder)

    print("Fichero escrito:"+file_path)

    # Obtener el nombre del fichero sin extensión
    nombre_fichero_sin_extension = os.path.splitext(os.path.basename(file_path))[0]

    wav_file = os.path.join(os.path.dirname(file_path), nombre_fichero_sin_extension + '.wav')

    print(wav_file)

    # Check if file is abc, mid, or xml
    if file_path.endswith(".abc"):
        # Crear rutas para los archivos .mid y .wav
        midi_file = uploads_folder + nombre_fichero_sin_extension + '.mid'
        midi_file = converter.abc_to_midi(file_path, midi_file)
        wav_file = converter.midi_to_wav(midi_file, wav_file)
    elif file_path.endswith(".mid"):
        wav_file = converter.midi_to_wav(file_path, wav_file)
    elif file_path.endswith(".xml"):
        midi_file = uploads_folder + nombre_fichero_sin_extension + '.mid'
        midi_file = converter.xml_to_midi(file_path,midi_file)
        wav_file = converter.midi_to_wav(midi_file, wav_file)
    else:
        raise Exception("File format not supported")

    return wav_file

def transform_wav_to_embedding_row(wav_file):
    # Extract embedding from a music file
    embedding = extractor.get_embedding(wav_file)
    print("Jukemir embedding: ")
    print(embedding)

    # Crear nombres de columnas basados en la longitud del embedding
    column_names = [f"embedding_jukemir_{i}" for i in range(len(embedding))]

    # Convertir el embedding en un dataframe y retornar
    embedding_df = pd.DataFrame([embedding], columns=column_names)
    return embedding_df

def classify_file(uploaded_file):
    # Supongamos que nuestro modelo nos da scores para tres clases: A, B y C
    # Estos son valores de ejemplo
    global classifier
    global converter

    # 1 - Transform abc, mid, or xml file to wav file
    wav_file = transform_file_to_wav(uploaded_file)
    print(wav_file)

    # 2 - Transform wav file to embedding row with columns embedding_mule_0, embedding_mule__1, etc
    df = transform_wav_to_embedding_row(wav_file)

    # 3 - Perform the inference
    pred = predict_model(classifier, df, raw_score=True)

    print(pred)

    # Get only the columns called prediction_score_*
    pred = pred.filter(regex="prediction_score_*")

    print(pred)

    # Get pred as a dictionary where keys are the columns names and values are the columns values
    pred_dict = pred.to_dict("records")[0]

    # Rename the keys of the dictionary to remove the prefix prediction_score_
    pred_dict = {key.replace("prediction_score_", ""): value for key, value in pred_dict.items()}

    return pred_dict


import gradio as gr

# Logo y título personalizado utilizando HTML
title_with_logo = """
    <div style='text-align: center;'>
        <div style='margin-bottom: 10px; font-weight: bold;'>
            IRISH TRADITIONAL MUSIC GENRE CLASSIFICATION
        </div>
        <div style='margin-bottom: 10px; font-weight: regular;'>
            Identifying Irish traditional music genres using latent audio representations
        </div>
        <div style='display: flex; justify-content: space-between; align-items: center;'>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://usal.es/files/logo_usal.png" alt="Logo" width="150" height="50" style="margin:10px;padding:10px;">
            </a>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://www.itma.ie/wp-content/themes/ITMA/images/itma-logo.svg" alt="Logo" width="150" height="50" style="margin:10px;padding:10px;">
            </a>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://cordis.europa.eu/images/logo/logo-ec-es.svg" alt="Logo" width="150" height="50" style="margin:10px;padding:10px;">
            </a>
        </div>
    </div>
"""

with gr.Blocks(title="ITMA Classifier", theme=gr.themes.Default(primary_hue="emerald")) as demo:
    iface = gr.Interface(
        fn=classify_file,
        inputs=gr.File(label="Upload .abc, .mid, or .xml file"),
        outputs=gr.Label(num_top_classes=N_GENRES),
        live=False,
        title=title_with_logo,
        allow_flagging="never"
    )

# Share the demo with a public link
demo.launch(share=True, inbrowser=True, debug=True)

Transformation Pipeline and Model Successfully Loaded
Loaded embedding model: 
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2fe059c8e7d2905b92.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/tmp/gradio/5c0ea9a1127c75807c002e88dc88285df1717565/tune1332setting13321.mid
Fichero escrito:./uploads/tune1332setting13321.mid
./uploads/tune1332setting13321.wav
./uploads/tune1332setting13321.wav
Downloading: 17% [1837137920 / 10288727721] bytes